# Robustness Results Viewer — Auto-detect
This notebook auto-finds the latest robustness run under any parent `runs/robustness_*` folder.

- Set `OVERRIDE` to a specific folder **only if** you want to force a different run.
- Otherwise it will scan your current directory and its parents for the newest `runs/robustness_*` automatically.


In [ ]:
from pathlib import Path
import pandas as pd, plotly.express as px
from IPython.display import display

# Optional manual override (set to a string path or keep as None)
OVERRIDE = None  # e.g. r"C:\\Users\\speed\\Desktop\\Forex CFD's system\\runs\\robustness_with_volcarry"

def find_all_robustness(start: Path):
    cands = []
    for base in [start] + list(start.parents):
        r = base / 'runs'
        if r.exists():
            cands += list(r.glob('robustness_*'))
    return sorted(cands, key=lambda p: p.stat().st_mtime)

def choose_root():
    here = Path.cwd()
    if OVERRIDE:
        p = Path(OVERRIDE)
        return p if (p / 'summary.csv').exists() else None
    cands = find_all_robustness(here)
    return cands[-1] if cands else None

root = choose_root()
if not root or not (root/'summary.csv').exists():
    display(pd.DataFrame({'hint':["No robustness results found.",
                                  "Run a sweep (e.g. --out_root runs\\robustness_smoketest)",
                                  "Or set OVERRIDE to your run folder path."]}))
else:
    print('Using robustness folder:', root)
    df = pd.read_csv(root / 'summary.csv')
    # Show top by Sharpe
    display(df.sort_values('sharpe', ascending=False).head())
    # Scatter: Sharpe vs MaxDD
    hover_cols = [c for c in ['lookbacks','w_tsmom','w_xsec','w_mr','w_volcarry','target_vol','vol_lookback','max_leverage','loo'] if c in df.columns]
    fig = pd.DataFrame()
    import plotly.express as px
    fig = px.scatter(df, x='maxdd', y='sharpe', color='target_vol', hover_data=hover_cols)
    fig.update_layout(title='Sharpe vs MaxDD (color=target_vol)')
    fig.show()
    # Pareto frontier (min |DD|, max Sharpe)
    import numpy as np
    dx = df['maxdd'].abs(); dy = df['sharpe']
    order = np.lexsort((-dy.values, dx.values))
    mask = [False]*len(df); best_s=-1e9; best_d=1e9
    for i in order:
        d=float(dx.iloc[i]); s=float(dy.iloc[i])
        if d<=best_d and s>=best_s:
            mask[i]=True; best_d=d; best_s=s
    pf = df[mask]
    fig2 = px.scatter(pf, x='maxdd', y='sharpe', color='target_vol', hover_data=hover_cols)
    fig2.update_layout(title='Pareto Frontier (min |DD|, max Sharpe)')
    fig2.show()
